In [1]:
from synthesizeme.personalrm.synthesizeme import SynthesizeMe
from synthesizeme.datasets import PrismDataset

/opt/miniconda3/envs/personal_rm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Collect User Feedback

In [2]:
dataset = PrismDataset()
train, val, test = dataset.get_user_data(dataset.get_user_ids()[67])
# train, val, test = dataset.get_user_data("user123")

In [3]:
data = train[:]
data.extend(val)

## View the data

In [4]:
data[0]

{'conversation_id': 'c3156',
 'user_id': 'user584',
 'context': array([{'content': "Hello, my 18 year old daughter wants to get gauges in her ears but I'm dead against it. What can I do?", 'role': 'user'}],
       dtype=object),
 'chosen': {'content': "As a parent, it's natural to have concerns about your child's appearance and health. Gauges, which are large ear piercings that stretch the earlobe, can be a permanent and visible change.\n\nHere are a few suggestions to help you address your daughter's request:\n\n1. Listen to her reasons: Encourage your daughter to explain why she wants gauges. Is it a fashion trend or does she have deeper reasons? Understanding her perspective can help you find a compromise or address her concerns in a more constructive way.\n\n2. Discuss the potential risks: Stretched ear piercings can lead to infection, tearing, and discomfort. Explain the potential complications and the importance of proper care and maintenance.\n\n3. Consider alternative options: 

## Train Personal Reward Model

In [5]:
# rm = SynthesizeMe(model_id='together_ai/meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', num_workers=64)
# rm = SynthesizeMe(model_id='gemini/gemini-2.0-flash', num_workers=64)
# rm = SynthesizeMe(model_id='gemini/gemini-2.5-flash-preview-05-20', num_workers=64)
# rm = SynthesizeMe(model_id='Qwen/Qwen3-8B', num_workers=64, model_url='http://future-hgx-1.stanford.edu:7420/v1')
# rm = SynthesizeMe(model_id='Qwen/Qwen3-30B-A3B', num_workers=64, model_url='http://future-hgx-1.stanford.edu:7410/v1')
# rm = SynthesizeMe(model_id='Qwen/Qwen3-32B', num_workers=64, model_url='http://future-hgx-1.stanford.edu:7450/v1')
# rm = DefaultJudge(model_id='Qwen/Qwen3-30B-A3B', num_workers=64, model_url='http://future-hgx-1.stanford.edu:7410/v1')
rm = SynthesizeMe(model_id='openai/gpt-4o-mini-2024-07-18', num_workers=64)

In [6]:
rm.fit(data)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]2025/05/30 11:09:14 WARNING dspy.primitives.module: There is a mismatch of python version between saved model and current environment. You saved with `python==3.12`, but now you have `python==3.10`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.
2025/05/30 11:09:14 WARNING dspy.primitives.module: There is a mismatch of dspy version between saved model and current environment. You saved with `dspy==2.6.12`, but now you have `dspy==2.6.24`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.
Tested 13 potential demo sets with persona.: 100%|██████████| 100.0/100 [00:32<00:00,  3.11it/s]                      


judge_persona.predict = Predict(StringSignature(conversation, first_completion, second_completion -> reasoning, preference
    instructions="Given a conversation and two completions from different models, alongside some prior judgements and a user persona, determine which completion the human judge is more likely to prefer.  Use any provided context as well as the provided persona to speculate about the personal preferences of the judge.  You are a personalized reward model for this user, so think carefully about what this user will like.\nThe user you are judging completions for has the FOLLOWING PERSONA: ===\nThe synthesized persona is a thoughtful and concerned parent who values open communication and understanding in family discussions. They are keen on supporting their child's autonomy while also wanting to ensure that their child makes informed decisions that consider potential long-term consequences. This persona is likely to engage in discussions that are empathetic and respect

In [7]:
rm.get_persona()

"The synthesized persona is a thoughtful and concerned parent who values open communication and understanding in family discussions. They are keen on supporting their child's autonomy while also wanting to ensure that their child makes informed decisions that consider potential long-term consequences. This persona is likely to engage in discussions that are empathetic and respectful, seeking to balance their concerns with their child's individuality. They appreciate nuanced responses that acknowledge the complexities of personal choices and societal norms, and they are open to exploring various perspectives to foster constructive dialogue. Overall, this persona reflects a commitment to nurturing their child's growth while being mindful of the implications of their choices."

In [8]:
# if rm.get_demos():
#     print(rm.get_demos()[0])

In [9]:
rm.evaluate(test)

{'mean': 0.3333333333333333,
 'lower_bound': 0.08333333333333326,
 'upper_bound': 0.5833333333333333,
 'results': [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0]}